In [1]:
import pandas as pd
from numpy import asarray
from numpy import zeros
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
df = pd.read_csv('open_response_filter.csv')
df_answers=pd.DataFrame(df['answer_text'])
df_test=df_answers
for index, row in df.iterrows():
    sentence=row['answer_text']
    sentence=sentence.replace("."," ")
    sentence=sentence.replace("+"," + ")
    sentence=sentence.replace("/"," / ")
    sentence=sentence.replace("*"," * ")
    sentence=sentence.replace("-"," - ")
    sentence=sentence.replace("("," ( ")
    sentence=sentence.replace(")"," ) ")
    sentence=sentence.replace("'"," ")
    sentence=sentence.replace("="," = ")
    sentence=sentence.replace(":"," : ")
    sentence=sentence.replace("<"," < ")
    sentence=sentence.replace(">"," > ")
    sentence=sentence.replace("^"," ^ ")
    sentence=sentence.replace(","," ")
    sentence=sentence.replace(u'\xa0', u' ')
    sentence=sentence.replace(u'\r\n', u'')
    sentence=sentence.replace('  ', ' ')
    sentence=sentence.replace('   ', ' ')
    
    sentence=sentence.lower()
    sentence=sentence.strip()
    df.set_value(index,'answer_text',sentence)

Using TensorFlow backend.


In [2]:
df_orig=pd.read_csv('open_response_filter.csv') #Loading it to just have questions accessible in the loop

In [3]:
#Filtering only these columns
df=df[['problem_id','answer_text','correct']]

In [4]:
df[0:5]

,problem_id,answer_text,correct
0,1276708,??,0.0
1,1276708,the frequency of the 38 in the table would be ...,1.0
2,1276708,there are 38 females that enrolled in band,1.0
3,1276708,38 means that there is 38 female students in b...,1.0
4,1276708,38 is the numder of females in band,1.0


In [5]:
#Trying to create dataframe with problem_id and the number of responses for each
max_count=df.groupby(['problem_id']).count()
max_count=max_count.sort_values(by=['answer_text'],ascending=False) #Sorting the dataframe by the #of resp
max_count['problem_id']=max_count.index

In [6]:
max_count[0:5]

,answer_text,correct,problem_id
problem_id,,,
1096508,69,69,1096508
1249297,56,56,1249297
1082516,52,52,1082516
1391885,48,48,1391885
1487480,44,44,1487480


In [7]:
#Getting the list of problem ids in this order
problem_list=max_count['problem_id'].tolist()

In [8]:
#Making the embedding from the glove to use in the next step in loops
embeddings_index = dict()
f = open('/home/avaratharaj/Glove/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400001 word vectors.


In [14]:
prob_acc=[]
import numpy as np
from sklearn.metrics import mean_squared_error
for problem in problem_list:
    print("Problem ID:",problem)
    print("Question:")
    print(list(set(df_orig.loc[df_orig['problem_id']==problem]['problem_text']))[0])
    problem_df=df.loc[df['problem_id']==problem] # Creating a dataframe for each problem
    docs=problem_df['answer_text'] #Getting the answers and labels to lists
    labels=problem_df['correct']
    docs=docs.tolist()
    labels=labels.tolist()
    
    print("Sample Answers:")
    print(docs[0:2])
    print("Correctness:")
    print(labels[0:2])
    #Removing responses greater than 900 which had junk values
    for i in docs:
        if(len(i)>1000):
            indx=docs.index(i)
            del docs[indx]
            del labels[indx]
    lens=[len(x) for x in docs] #Getting length of each response into a list
    
    #Getting a tokenizer ready
    t = Tokenizer()
    t.fit_on_texts(docs)
    vocab_size = len(t.word_index) + 1
    # integer encode the documents
    encoded_docs = t.texts_to_sequences(docs)
    #print(encoded_docs)
    # pad documents to a max length of the words
    max_length = max(lens)
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
    #print(padded_docs)
    
    # create a weight matrix for words in training docs of this problem_ID
    embedding_matrix = zeros((vocab_size, 100))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    
    
    #Create the embedding 
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max(lens), trainable=False)
    pd_array=np.array(padded_docs)
    lab_arr=np.array(labels)
    #define model
    model = Sequential()
    model.add(e)
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    kf = KFold(n_splits=5)
    kf.get_n_splits(pd_array)
    mean_mse=[]
    mean_accuracy=[]
    for train_index, test_index in kf.split(pd_array):

        X_train, X_val = pd_array[train_index], pd_array[test_index]
        y_train, y_val = lab_arr[train_index], lab_arr[test_index]
        model.fit(X_train, y_train, epochs=30, verbose=0)
        # evaluate the model
        loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
        print('Accuracy: %f' % (accuracy*100))
        y_pred_val=model.predict(X_val)
        #print("ROC:",roc_auc_score(y_val,y_pred_val))

        print("MSE:",mean_squared_error(y_val,y_pred_val))
        mean_accuracy.append(accuracy)
        mean_mse.append(mean_squared_error(y_val,y_pred_val))
    mean_accuracy=np.array(mean_accuracy)
    mean_mse=np.array(mean_mse)
    acc=mean_accuracy.mean()
    mse=mean_mse.mean()
    
    
    
    prob_acc.append([problem,len(docs),acc*100,mse])

Problem ID: 1096508
Question:
Decide where each of the following events would be located on the scale below. Place the letter for each event on the appropriate place on the probability scale. 
 
   
 The numbers from 1 to 10 are written on small pieces of paper and placed in a bag. A piece of paper will be drawn from the bag. 
   
 A. A piece of paper with a 𝟓 is drawn from the bag. 
   

Sample Answers:
['a - unlikely b - equally likely c - impossible d - likely e - unlikely', 'a = unlikely  b = equal c = impossible  d = likely  e = unlikely']
Correctness:
[1.0, 1.0]
Accuracy: 64.285713
MSE: 0.0451795026693
Accuracy: 78.571427
MSE: 0.00382332888698
Accuracy: 78.571427
MSE: 0.00576916144615
Accuracy: 57.142860
MSE: 0.00782619345545
Accuracy: 30.769232
MSE: 0.00375084786707
Problem ID: 1249297
Question:
9. A parallelogram has an area of 20.3 sq. cm and a base of 2.5 cm. Write an equation that relates the area to the base and height, h. Do not include units. 
 Write the equation using th

In [15]:
prob_acc=pd.DataFrame(prob_acc)
prob_acc.columns=['problem_id','Number of responses','Accuracy','MSE']
pd.options.display.max_rows = 10000
prob_acc

,problem_id,Number of responses,Accuracy,MSE
0,1096508,69,61.868132,0.013270
1,1249297,56,98.181819,0.020498
2,1082516,52,96.363637,0.045807
3,1391885,47,38.888890,0.010113
4,1487480,44,67.777779,0.027609
5,1415683,42,41.666667,0.023138
6,1415679,41,61.666667,0.028724
7,1383722,40,100.000000,0.046471
8,1383720,39,97.500000,0.054989
9,1391484,39,72.142857,0.017184
